In [24]:
%matplotlib inline
import numpy as np
import json

In [33]:
#ref http://stackoverflow.com/questions/8955448/save-load-scipy-sparse-csr-matrix-in-portable-data-format
def save_sparse_csr(filename,array):
    np.savez(filename,data = array.data ,indices=array.indices,
             indptr =array.indptr, shape=array.shape )

def load_sparse_csr(filename):
    loader = np.load(filename)
    return csr_matrix((  loader['data'], loader['indices'], loader['indptr']),shape = loader['shape'])

In [34]:
co_author_matrix=load_sparse_csr("../data/co-author-matrix.npz")
with open('../data/co-author-index.json', 'r') as f:
    aid2aname=json.load(f)
aid2aname=dict((int(k), v) for k, v in aid2aname.iteritems())

In [37]:
with open('../data/co-author-index.json', 'r') as f:
    aid2aname=json.load(f)
aid2aname=dict((int(k), v) for k, v in aid2aname.iteritems())

In [38]:
def top_k(C,c_id,cid2cname,topk=5):
    norms = np.apply_along_axis(np.linalg.norm, 0, C)
    C_norm = C / norms.reshape(1,-1)
    c_vec=C_norm[c_id]
    sim = np.dot(C_norm,c_vec)
    nearest = (-sim).argsort()[1:topk+1]
    results=[(cid2cname[nearest[k]],sim[nearest[k]]) for k in xrange(topk)]
    return results

def top_k_vec(C,vec,cid2cname,topk=5):
    C_norm=normalize(C)
    vec_norm=vec/np.linalg.norm(vec)
    sim = np.dot(C_norm,vec_norm)
    nearest = (-sim).argsort()[0:topk]
    results=[(cid2cname[nearest[k]],sim[nearest[k]]) for k in xrange(topk)]
    return results

In [50]:
np_node_embeddings=np.load('../results/node_embeddings.npz')['arr_0']

In [52]:
top_k(np_node_embeddings,10937,aid2aname)

[(u'rohan puttagunta', 0.0031667058),
 (u'caroline suen', 0.0030133841),
 (u'pararth shah', 0.0028435378),
 (u'nilesh dalvi', 0.0028338402),
 (u'ashton anderson', 0.0028116952)]

In [51]:
top_k(np_node_embeddings,71929,aid2aname)

[(u'wilhelm hasselbring', 0.0027626124),
 (u'gunter saake', 0.0026860675),
 (u'kaiuwe sattler', 0.0026787217),
 (u'stefan conrad', 0.0026114527),
 (u'm strassler', 0.0025326342)]

In [ ]:
#If you want to restore from ckpt files, this might be helpful....


# from gensim import corpora
# import numpy as np
# import unicodecsv as csv
# import tensorflow as tf
# import math
# import os,sys
# import random
# from scipy.sparse import csr_matrix
# from tqdm import tqdm
# import json

# #Computational Graph Definition
# tf.reset_default_graph()#remove this if not ipython notebook

# num_nodes=adj_mat_csr_sparse.shape[0]
# context_size=16
# batch_size = None
# embedding_size = 200 # Dimension of the embedding vector.
# num_sampled = 64 # Number of negative examples to sample.

# global_step = tf.Variable(0, name='global_step', trainable=False)

# # Parameters to learn
# node_embeddings = tf.Variable(tf.random_uniform([num_nodes, embedding_size], -1.0, 1.0))

# #Fixedones
# biases=tf.zeros([num_nodes])

# # Input data and re-orgenize size.
# with tf.name_scope("context_node") as scope:
#     #context nodes to each input node in the batch (e.g [[1,2],[4,6],[5,7]] where batch_size = 3,context_size=3)
#     train_context_node= tf.placeholder(tf.int32, shape=[batch_size,context_size],name="context_node")
#     #orgenize prediction labels (skip-gram model predicts context nodes (i.e labels) given a input node)
#     #i.e make [[1,2,4,6,5,7]] given context above. The redundant dimention is just for restriction on tensorflow API.
#     train_context_node_flat=tf.reshape(train_context_node,[-1,1])
# with tf.name_scope("input_node") as scope:
#     #batch input node to the network(e.g [2,1,3] where batch_size = 3)
#     train_input_node= tf.placeholder(tf.int32, shape=[batch_size],name="input_node")
#     #orgenize input as flat. i.e we want to make [2,2,2,1,1,1,3,3,3] given the  input nodes above
#     input_ones=tf.ones_like(train_context_node)
#     train_input_node_flat=tf.reshape(tf.mul(input_ones,tf.reshape(train_input_node,[-1,1])),[-1])

# # Model.
# with tf.name_scope("loss") as scope:
#     # Look up embeddings for words.
#     node_embed = tf.nn.embedding_lookup(node_embeddings, train_input_node_flat)
#     # Compute the softmax loss, using a sample of the negative labels each time.
#     loss_node2vec = tf.reduce_mean(tf.nn.sampled_softmax_loss(node_embeddings,biases,node_embed,train_context_node_flat, num_sampled, num_nodes))
#     loss_node2vec_summary = tf.scalar_summary("loss_node2vec", loss_node2vec)

# # Initializing the variables
# init = tf.initialize_all_variables()

# # Add ops to save and restore all the variables.
# saver = tf.train.Saver(max_to_keep=20)

# merged = tf.merge_all_summaries()

# with tf.Session() as sess:
#     # Restore variables from disk.
#     log_dir="../log1/"
#     global_step=30001
#     model_path=log_dir+"model.ckpt-%d"%global_step
#     saver.restore(sess, model_path)
#     print("Model restored.")
#     node_embeddings_=sess.run(node_embeddings)